In [2]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install evaluate
!pip install sacrebleu

  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 kB 7.8 MB/s eta 0:00:00
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.9/288.9 kB 16.6 MB/s eta 0:00:00
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 36.1 MB/s eta 0:00:00a 0:00:01
  Using cached fsspec-2023.5.0-py3-none-any.whl (160 kB)
  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.9/336.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 34.3 MB/s eta 0:00:0000:010:01
  Using cached multiprocess-0.70.14-py310-none-any.whl (134 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.6/22.6 MB 

In [1]:
import tensorflow as tf

In [2]:
import torch

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
import numpy as np

In [4]:
import evaluate
metric = evaluate.load("sacrebleu")

In [5]:
model_dir = "/Users/baesangjun/Desktop/SWProject/result2"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
max_token_length = 64

In [6]:
import multiprocessing

en_ko = load_dataset("bongsoo/news_talk_en_ko")
en_ko.set_format(type="pandas")
df = en_ko["train"][:]
example_0 = list(df.columns)
example_0_df = pd.DataFrame({col: [value] for col, value in zip(('en', 'ko'), example_0)})
df.columns = ('en', 'ko')
en_ko_df = pd.concat([example_0_df, df],).reset_index(drop=True)
dataset = Dataset.from_pandas(en_ko_df)
# 각 데이터 셋의 샘플수를 정한다.
num_train = 1200000
num_valid = 90000
num_test = 10000

en_ko_df_train = en_ko_df.iloc[:num_train]
en_ko_df_valid = en_ko_df.iloc[num_train:num_train+num_valid]
en_ko_df_test = en_ko_df.iloc[-num_test:]

en_ko_df_train.to_csv("train.tsv", sep='\t', index=False)
en_ko_df_valid.to_csv("valid.tsv", sep='\t', index=False)
en_ko_df_test.to_csv("test.tsv", sep='\t', index=False)

data_files = {"train": "train.tsv", "valid": "valid.tsv", "test": "test.tsv"}
dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")


Found cached dataset csv (/Users/baesangjun/.cache/huggingface/datasets/bongsoo___csv/bongsoo--news_talk_en_ko-e7f00bc8f76f18d5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/baesangjun/.cache/huggingface/datasets/csv/default-0c7f67879a7f0b04/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

# tokenizer = AutoTokenizer.from_pretrained(model_ckpt) 

def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['en'],
                             text_target=examples['ko'], 
                             max_length=max_token_length, truncation=True)
    
    return model_inputs

NUM_CPU = multiprocessing.cpu_count() 


tokenized_datasets = dataset.map(convert_examples_to_features, 
                                 batched=True, 
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서
                                 # 아래서 콜레이터가 패딩을 못해서 에러남
                                 remove_columns=dataset["train"].column_names,
                                 num_proc=NUM_CPU) 

tokenized_datasets

Map (num_proc=8):   0%|          | 0/1200000 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/90000 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1200000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})

In [8]:
from torch.utils.data import DataLoader

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

In [9]:
test_dataloader_iter = iter(test_dataloader)
test_batch = next(test_dataloader_iter)
test_batch.keys()
test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }
koreans = model.generate(
    **test_input,
    max_length=max_token_length,
    num_beams=5,
)
labels =  np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)
eng_sents = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True)[0:1000]
references = tokenizer.batch_decode(labels, skip_special_tokens=True)[0:1000]
preds = tokenizer.batch_decode( koreans, skip_special_tokens=True )[0:1000]

for s in zip(eng_sents, references, preds):
    print('English   :', s[0])
    print('Reference :', s[1])
    print('Translated:', s[2])
    print('\n')


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


English   : I already placed it on the express order yesterday.
Reference : 제가 어제 이미 급송 주문으로 넣어두었습니다.
Translated: 저는 어제 이미 급행 주문에 넣었습니다.


English   : Did they say the items could arrive on time?
Reference : 물건들은 제날짜에 도착할 수 있다고 하던가요?
Translated: 그 물건들이 제때 도착할 수 있다고 했나요?


English   : Yes, all the items will arrive the day before the new employees arrive at work.
Reference : 네, 물건들은 새로운 직원들이 출근하기 전날에 모두 도착할 것입니다.
Translated: 네, 신입사원이 출근하기 하루 전에 모든 물품이 도착합니다.


English   : We need to visit their offices and do an assessment as this is potentially a large-scale project.
Reference : 이번에는 규모가 큰 프로젝트이니, 그들의 사무실을 방문해서 조사해야 해요.
Translated: 우리는 대규모 프로젝트일 가능성이 있는 만큼 그들의 사무실을 방문해서 평가를 해야 합니다.


English   : Do you need me to do anything in regard to the project?
Reference : 이 건에 대해 제가 무언가 해야 할 것이 있나요?
Translated: 프로젝트와 관련하여 내가 무엇을 해야 하나요?


English   : Yes, I really need your expertise.
Reference : 네, 저는 정말로 당신의 전무 지식이 필요해요.
Translated: 네, 나는 정말 당신의 전문지식이 필요해요.


English   : Yes, let me know what 

In [12]:
score = []
for i in range(len(preds)):
    s = metric.compute(predictions=[preds[i]], references=[references[i]])
    score.append(s['score'])
print('BLEU :' , np.mean(score))

BLEU : 15.007608917450117


32